### Imports

In [1]:
# Imports
import pickle
import numpy as np
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from pandas_datareader import data as web

# Part One - Data Acquisition and Cleaning

## Web Scraping

Web scraping the list of S&P 500 companies from website https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
As of January 20, 2018. NOTE: Saved a local copy 'data/201801201706 - List of S&P 500 companies - Wikipedia.html'

In [ ]:
# https://stackoverflow.com/questions/42225204/use-pandas-to-get-multiple-tables-from-webpage


url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
html_table = urllib.request.urlopen(url).read()

# fix HTML
soup = BeautifulSoup(html_table, "html.parser")
# warn! id ratings-table is your page specific
for table in soup.findChildren(attrs={'id': 'ratings-table'}): 
    for c in table.children:
        if c.name in ['tbody', 'thead']:
            c.unwrap()

list_df = pd.read_html(str(soup), flavor="bs4")
#len(list_df[0])

### Debug

In [ ]:
# List of dataframes.
# The first Dataframe in the list is the conversion of the HTML table
# to a Pandas Dataframe. The first one is the one we care about. It is the 
# S&P 500 Component Stocks.
type(list_df[0])

In [ ]:
# Randomly sample axis, in this case None looks like the rows.
list_df[0].sample(10,
                  random_state = None)

### Pass Pandas Dataframe we care about to a more intuitive variable name.

In [ ]:
df_sp500_component_stocks_raw_data = list_df[0]

### Save raw data to .csv

In [ ]:
file_name_web_scrap_raw_data = 'data/sp500_component_stocks_raw_data_201801201730.csv'
df_sp500_component_stocks_raw_data.to_csv(file_name_web_scrap_raw_data)

### Work from loaded raw data instead of web scrapping each time.

 The web site can change over time where as we are data locking from this point on and using the saved/cached version. If we need to web scrap again, run the first section. NOTE: The web address may change and/or the format may change.

### Load raw data from file

In [ ]:
# Using same variable name.
df_sp500_component_stocks_raw_data = pd.read_csv( file_name_web_scrap_raw_data )

### Debug

In [ ]:
df_sp500_component_stocks_raw_data.sample(10, random_state = None)

### Clean the data. Make the first row the head for the columns.

In [ ]:
# Make the first row the header column
# NOTE: This does not get rid of the row.
df_sp500_component_stocks_cleaned = df_sp500_component_stocks_raw_data.copy()
df_sp500_component_stocks_cleaned.columns = df_sp500_component_stocks_cleaned.iloc[0]

# Re-index and drop the first row.
df_sp500_component_stocks_cleaned = df_sp500_component_stocks_cleaned.reindex(df_sp500_component_stocks_cleaned.index.drop(0))

# # Keep columns of interest.
# # https://stackoverflow.com/questions/14940743/selecting-excluding-sets-of-columns-in-pandas
# columns_to_keep = ['GICS Sector', 'GICS Sub Industry']
# df_sp500_component_stocks_cleaned = df_sp500_component_stocks_cleaned[columns_to_keep]

# This is the main industry, select only the rows for ticker symbols for the main industry.
# NOTE: The main industry is System Software (i.e. operating systems companies like Red Hat or Microsoft.)
#       Not just broad 
#
# https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas
#df_tickers_for_information_technology = df_sp500_component_stocks_cleaned.loc[df_sp500_component_stocks_cleaned['GICS Sector'] == 'Information Technology']
#
# not plural and ignore case:
# https://stackoverflow.com/questions/32616261/filtering-pandas-dataframe-rows-by-contains-str
df_tickers_for_software = df_sp500_component_stocks_cleaned.loc[df_sp500_component_stocks_cleaned['GICS Sub Industry'].str.contains('software', case=False)]

# Related industry, semiconductors
# not plural and ignore case:
# https://stackoverflow.com/questions/32616261/filtering-pandas-dataframe-rows-by-contains-str
df_tickers_for_semiconductors = df_sp500_component_stocks_cleaned.loc[df_sp500_component_stocks_cleaned['GICS Sub Industry'].str.contains('semiconductor', case=False)]


# Reset index.
# NOTE: drop = True means do not make a new index and keep old.
#       inplace = True means update this variable and not return a copy
#                      leaving original intact.
df_tickers_for_software.reset_index(drop = True,
                              inplace = True)
df_tickers_for_semiconductors.reset_index(drop = True,
                                          inplace = True)

# This is the related industry.
# NOTE: The related industry is semiconductors.


### Debug

In [ ]:
#df_sp500_component_stocks_cleaned.sample(10, random_state = None)
#df_sp500_component_stocks_cleaned.head(10)
#df_sp500_component_stocks_cleaned.tail(10)
#df_tickers_for_software
df_tickers_for_semiconductors


### Save clean data to .csv

In [ ]:
df_sp500_component_stocks_cleaned.to_csv( 'data/sp500_component_stocks_cleaned_data_201801201826.csv' )
df_tickers_for_software.to_csv( 'data/tickers_main_industry_software_201801201826.csv')
df_tickers_for_semiconductors.to_csv( 'data/tickers_related_industry_semiconductor_201801201826.csv')

### Load data derived from web data scraping.

In [ ]:
df_tickers_for_software = pd.read_csv( 'data/tickers_main_industry_software_201801201826.csv',
                 
                                       # Use the first column as the index
                                       index_col = 0)

df_tickers_for_semiconductors = pd.read_csv( 'data/tickers_related_industry_semiconductor_201801201826.csv',
                 
                                             # Use the first column as the index
                                             index_col = 0)

### Debug

In [ ]:
df_tickers_for_software

In [ ]:
df_tickers_for_semiconductors

# Part Two - Data Processing

### Get ticker symbols for the main industry and related industry
### From Rubric
### Step 1: List down all stocks in the industry

In [ ]:
# Get the value from the column, which is a Pandas Series, and convert to a Python List.
list_tickers_for_software = df_tickers_for_software['Ticker symbol'].tolist()
list_tickers_for_semiconductors = df_tickers_for_semiconductors['Ticker symbol'].tolist()

### Debug

In [345]:
list_tickers_for_software

['ADBE',
 'ADP',
 'ADSK',
 'AKAM',
 'ANSS',
 'ATVI',
 'CA',
 'CDNS',
 'CRM',
 'CTXS',
 'EA',
 'EBAY',
 'FB',
 'FIS',
 'FISV',
 'GOOG',
 'GOOGL',
 'INTU',
 'MA',
 'MSFT',
 'NFLX',
 'NTAP',
 'ORCL',
 'PAYX',
 'RHT',
 'SNPS',
 'SYMC',
 'TSS',
 'V',
 'VRSN',
 'WU']

In [346]:
list_tickers_for_semiconductors

['ADI',
 'AMAT',
 'AMD',
 'AVGO',
 'INTC',
 'KLAC',
 'LRCX',
 'MCHP',
 'MU',
 'NVDA',
 'QCOM',
 'QRVO',
 'SWKS',
 'TXN',
 'XLNX']

### From Rubric
### Step 2: Collect last one year stock price data for these stocks.

#### Main Industry

In [ ]:
dict_tickers_for_software = {}

for ticker in list_tickers_for_software:
    dict_tickers_for_software[ticker] = web.get_data_quandl(ticker,
                                                            start = '1/19/2017',
                                                            end = '1/19/2018')

# df_temp_a = []
# for ticker in list_tickers_for_software:
    
#     # We get all data available
#     df_temp_a = web.get_data_quandl(ticker)
    
#     # The data is dated latest first.
#     # We only want a year, so keep only 252 days.
#     dict_tickers_for_software[ticker]  = df_temp_a[:252]
    

In [ ]:
dict_tickers_for_semiconductors = {}

for ticker in list_tickers_for_semiconductors:
    dict_tickers_for_semiconductors[ticker] = web.get_data_quandl(ticker,
                                                                  start = '1/19/2017',
                                                                  end = '1/19/2018')

# for ticker in list_tickers_for_semiconductors:
    
#     # We get all data available
#     df_temp_a = web.get_data_quandl(ticker)
    
#     # The data is dated latest first.
#     # We only want a year, so keep only 252 days.
#     dict_tickers_for_semiconductors[ticker]  = df_temp_a[:252]

### Get SPX, the ticker that represents the S&P 500 to compare market returns

In [81]:
spx_ticker = web.DataReader( 'SPX',
                       data_source = 'yahoo',
                       start = '1/19/2017',
                       end = '1/19/2018')

In [82]:
spx_ticker

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-19,0.006,0.006,0.006,0.006,0.006,0
2017-01-20,0.045,0.045,0.045,0.045,0.045,0
2017-01-23,0.045,0.045,0.045,0.045,0.045,58000
2017-01-24,0.045,0.045,0.045,0.045,0.045,19000
2017-01-25,0.045,0.050,0.045,0.045,0.045,204250
2017-01-26,0.050,0.050,0.045,0.045,0.045,46000
2017-01-27,0.045,0.045,0.045,0.045,0.045,12000
2017-01-30,0.006,0.006,0.006,0.006,0.006,0
2017-01-31,0.040,0.040,0.040,0.040,0.040,8000


### Debug

In [ ]:
# #dict_tickers_for_software['MSFT']['2017-01-19']
# df_temp_a = dict_tickers_for_software['MSFT'].reset_index()

In [ ]:
# #df_temp_a.iloc[:252]
# # df_temp_b = df_temp_a.set_index( df_temp_a.iloc[:252]['Date'],
# #                                  drop = True )
# df_temp_c = df_temp_a.iloc[:252]
# #df_temp_b = dict_tickers_for_software['MSFT'].iloc['1/19/2017':'1/19/2018']

In [ ]:
# df_temp_a = dict_tickers_for_software['MSFT']

In [ ]:
# df_temp_a.iloc[:252]

In [ ]:
dict_tickers_for_software.keys()

In [ ]:
dict_tickers_for_software['MSFT']dict_tickers_for_semiconductors

In [ ]:
dict_tickers_for_semiconductors.keys()

In [ ]:
dict_tickers_for_semiconductors['INTC']

### Save Data to Pickle

In [ ]:
# https://stackoverflow.com/questions/11641493/how-to-cpickle-dump-and-load-separate-dictionaries-to-the-same-file

filename = 'data/dict_tickers_for_software_201801201933.pickle'
with open(filename,'wb') as fp:
    pickle.dump(dict_tickers_for_software,fp)
    
filename = 'data/dict_tickers_for_semiconductors_201801201933.pickle'
with open(filename,'wb') as fp:
    pickle.dump(dict_tickers_for_semiconductors,fp)

### Loading Data from Pickle

In [26]:
# https://stackoverflow.com/questions/11641493/how-to-cpickle-dump-and-load-separate-dictionaries-to-the-same-file

filename_software = 'data/dict_tickers_for_software_201801201933.pickle'
filename_semiconductor = 'data/dict_tickers_for_semiconductors_201801201933.pickle'

with open(filename_software,'rb') as fp:
    dict_tickers_for_software=pickle.load(fp)
    
with open(filename_semiconductor,'rb') as fp:
    dict_tickers_for_semiconductors=pickle.load(fp)

### Debug

In [ ]:
dict_tickers_for_software.keys()

In [ ]:
dict_tickers_for_software['MSFT']

In [ ]:
dict_tickers_for_semiconductors.keys()

In [ ]:
dict_tickers_for_semiconductors['INTC']

### Create Pandas Panels to have multiple pages for Dataframes.

In [3]:
# https://www.tutorialspoint.com/python_pandas/python_pandas_panel.htm

dp_tickers_for_software = pd.Panel(dict_tickers_for_software)
dp_tickers_for_semiconductors = pd.Panel(dict_tickers_for_semiconductors)

### Debug

In [4]:
dir(dp_tickers_for_software)

['ADBE',
 'ADP',
 'ADSK',
 'AKAM',
 'ANSS',
 'ATVI',
 'CA',
 'CDNS',
 'CRM',
 'CTXS',
 'EA',
 'EBAY',
 'FB',
 'FIS',
 'FISV',
 'GOOG',
 'GOOGL',
 'INTU',
 'MA',
 'MSFT',
 'NFLX',
 'NTAP',
 'ORCL',
 'PAYX',
 'RHT',
 'SNPS',
 'SYMC',
 'TSS',
 'V',
 'VRSN',
 'WU',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixo

In [5]:
dp_tickers_for_software.items

Index(['ADBE', 'ADP', 'ADSK', 'AKAM', 'ANSS', 'ATVI', 'CA', 'CDNS', 'CRM',
       'CTXS', 'EA', 'EBAY', 'FB', 'FIS', 'FISV', 'GOOG', 'GOOGL', 'INTU',
       'MA', 'MSFT', 'NFLX', 'NTAP', 'ORCL', 'PAYX', 'RHT', 'SNPS', 'SYMC',
       'TSS', 'V', 'VRSN', 'WU'],
      dtype='object')

### Save to Excel

Saving to Excel allows to use each page as a ticker symbol as a stock.

In [6]:
dp_tickers_for_software.to_excel('data/tickers_main_industry_software_201801211359.xls')
dp_tickers_for_semiconductors.to_excel('data/tickers_related_industry_semiconductor_201801211359.xls')

In [83]:
spx_ticker.to_excel('data/ticker_spx_201801281357.xls')

### Load from Excel

There does not seem to have a method to load an Excel file with multiple pages back into a Pandas Panel. The workaround is to load the Excel file as and Excel file object, find the sheet names (Excel pages,) and reconstruct a Pandas Panel.

In [2]:
# https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook

# Reconstruct Pandas Panel for Main industry Software:
xls_tickers_for_software = pd.ExcelFile('data/tickers_main_industry_software_201801211359.xls')

# .sheet_names is a property, not a method
list_tickers_for_software = xls_tickers_for_software.sheet_names

dict_tickers_for_software = {}

for ticker in list_tickers_for_software:

    dict_tickers_for_software[ticker] = pd.read_excel( xls_tickers_for_software,
                                                       ticker,
                                                       index_col = 'Date')
    
#-------------------------------

# Reconstruct Pandas Panel for related industry semiconductors:
xls_tickers_for_semiconductors = pd.ExcelFile('data/tickers_related_industry_semiconductor_201801211359.xls')

# .sheet_names is a property, not a method
list_tickers_for_semiconductors = xls_tickers_for_semiconductors.sheet_names

dict_tickers_for_semiconductors = {}

for ticker in list_tickers_for_semiconductors:

    dict_tickers_for_semiconductors[ticker] = pd.read_excel( xls_tickers_for_semiconductors,
                                                             ticker,
                                                             index_col = 'Date')

#-------------------------------
    
# Create Pandas Panels to have multiple pages for Dataframes.
# https://www.tutorialspoint.com/python_pandas/python_pandas_panel.htm
dp_tickers_for_software = pd.Panel(dict_tickers_for_software)
dp_tickers_for_semiconductors = pd.Panel(dict_tickers_for_semiconductors)

In [86]:
df_spx = pd.read_excel( 'data/ticker_spx_201801281357.xls',
               sheet_name = 0,
               index_col = 'Date')

In [87]:
df_spx

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-19,0.006,0.006,0.006,0.006,0.006,0
2017-01-20,0.045,0.045,0.045,0.045,0.045,0
2017-01-23,0.045,0.045,0.045,0.045,0.045,58000
2017-01-24,0.045,0.045,0.045,0.045,0.045,19000
2017-01-25,0.045,0.050,0.045,0.045,0.045,204250
2017-01-26,0.050,0.050,0.045,0.045,0.045,46000
2017-01-27,0.045,0.045,0.045,0.045,0.045,12000
2017-01-30,0.006,0.006,0.006,0.006,0.006,0
2017-01-31,0.040,0.040,0.040,0.040,0.040,8000


### Debug

In [9]:
dir(xls_tickers_for_software)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__fspath__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_io',
 '_parse_excel',
 '_should_parse',
 'book',
 'close',
 'io',
 'parse',
 'sheet_names']

In [10]:
xls_tickers_for_software.sheet_names

['ADBE',
 'ADP',
 'ADSK',
 'AKAM',
 'ANSS',
 'ATVI',
 'CA',
 'CDNS',
 'CRM',
 'CTXS',
 'EA',
 'EBAY',
 'FB',
 'FIS',
 'FISV',
 'GOOG',
 'GOOGL',
 'INTU',
 'MA',
 'MSFT',
 'NFLX',
 'NTAP',
 'ORCL',
 'PAYX',
 'RHT',
 'SNPS',
 'SYMC',
 'TSS',
 'V',
 'VRSN',
 'WU']

In [12]:
dict_tickers_for_software.keys()

dict_keys(['ADBE', 'ADP', 'ADSK', 'AKAM', 'ANSS', 'ATVI', 'CA', 'CDNS', 'CRM', 'CTXS', 'EA', 'EBAY', 'FB', 'FIS', 'FISV', 'GOOG', 'GOOGL', 'INTU', 'MA', 'MSFT', 'NFLX', 'NTAP', 'ORCL', 'PAYX', 'RHT', 'SNPS', 'SYMC', 'TSS', 'V', 'VRSN', 'WU'])

In [13]:
dict_tickers_for_semiconductors.keys()

dict_keys(['ADBE', 'ADP', 'ADSK', 'AKAM', 'ANSS', 'ATVI', 'CA', 'CDNS', 'CRM', 'CTXS', 'EA', 'EBAY', 'FB', 'FIS', 'FISV', 'GOOG', 'GOOGL', 'INTU', 'MA', 'MSFT', 'NFLX', 'NTAP', 'ORCL', 'PAYX', 'RHT', 'SNPS', 'SYMC', 'TSS', 'V', 'VRSN', 'WU'])

In [16]:
dp_tickers_for_software.items

Index(['ADBE', 'ADP', 'ADSK', 'AKAM', 'ANSS', 'ATVI', 'CA', 'CDNS', 'CRM',
       'CTXS', 'EA', 'EBAY', 'FB', 'FIS', 'FISV', 'GOOG', 'GOOGL', 'INTU',
       'MA', 'MSFT', 'NFLX', 'NTAP', 'ORCL', 'PAYX', 'RHT', 'SNPS', 'SYMC',
       'TSS', 'V', 'VRSN', 'WU'],
      dtype='object')

In [17]:
dp_tickers_for_semiconductors.items

Index(['ADI', 'AMAT', 'AMD', 'AVGO', 'INTC', 'KLAC', 'LRCX', 'MCHP', 'MU',
       'NVDA', 'QCOM', 'QRVO', 'SWKS', 'TXN', 'XLNX'],
      dtype='object')

### From Rubric
### Step 3: Calculate the historical distance measure between all the possible pairs of stocks.

First need to make Pandas DataFrame for both main industry (software) and related industry (semiconductors) with the ticker symbol and closing price.

NOTE: Using Adjusted Closing Price to take into account stock splits and dividends. https://www.investopedia.com/terms/a/adjusted_closing_price.asp

Simple trading strategy
Illiquid stocks are removed from the investment universe. Cumulative total return index is then created for each stock (dividends included) and starting price during formation period is set to $1 (price normalization). Pairs are formed over a twelve-month period (formation period) and are then traded in next six-month period (trading period). The matching partner for each stock is found by looking for the security that minimizes the sum of squared deviations between two normalized price series. Top 20 pairs with the smallest historical distance measure are then traded and long-short position is opened when pair prices have diverged by two standard deviations and the position is closed when prices revert back.

https://quantpedia.com/Screener/Details/12

In [4]:
# https://quantpedia.com/Screener/Details/12
# https://stackoverflow.com/questions/18062135/combining-two-series-into-a-dataframe-in-pandas

# Going to concat multiple Panda series into a Panda DataFrame
list_of_series_for_software = []
dict_of_dataframes_modified_for_software = {}
for ticker in dp_tickers_for_software.items:

    df_temp_e = pd.DataFrame()
        
    #----------------------------------------
    
    # Getting adjusted close. It takes into account stock split and
    # dividend pay outs.
    # https://www.investopedia.com/terms/a/adjusted_closing_price.asp
    #
    # Where price includes reinvested dividends from paper Pair Trading.
    # Section 2.1 - Pairs Formation Page 11.
    # This means we do NOT include dividends as cumulative returns.
    df_temp_e['AdjClose'] = dp_tickers_for_software[ticker]['AdjClose']
    df_temp_e['ExDividend'] = dp_tickers_for_software[ticker]['ExDividend']
   
    # Forward fill then backward fill data.
    df_temp_e = df_temp_e.ffill().bfill()

    # Adjusted close normalized by dividing by the price of day 1 of the formation period.
    df_temp_e['AdjClose_Normalized']  = df_temp_e['AdjClose'] / df_temp_e.iloc[0]['AdjClose']
    
    # Get the daily returns
    # https://stackoverflow.com/questions/20000726/calculate-daily-returns-with-pandas-dataframe
    df_temp_e['daily_returns'] = df_temp_e['AdjClose'] - df_temp_e['AdjClose'].shift(1)
   
    # Fill all not a number (NaN) with 0.
    df_temp_e['daily_returns'] = df_temp_e['daily_returns'].fillna(0.0)


    # Get cumulative returns
    df_temp_e['cumulative_returns'] = df_temp_e['daily_returns'].cumsum()
    
    # Add dividend pay out.
    df_temp_e['cumulative_returns_with_dividends'] = df_temp_e['cumulative_returns'] + df_temp_e['ExDividend']
    
    # Normalized cumulative returns
    df_temp_e['cumulative_returns_normalized'] = df_temp_e['cumulative_returns'] / df_temp_e.iloc[0]['AdjClose']
    
    # Normalized cumulative returns by dividing by the price of day 1 of the formation period.
    df_temp_e['cumulative_returns_with_dividends_normalized'] = df_temp_e['cumulative_returns_with_dividends'] / df_temp_e.iloc[0]['AdjClose']
    
    # Forward fill then backward fill data.
    df_temp_e = df_temp_e.ffill().bfill()

    
    # Gather data for Debugging
    df_temp_f = df_temp_e.copy()
    dict_of_dataframes_modified_for_software[ticker] = df_temp_f
    
    # Copy only the Pandas Series
    #
    # Where price includes reinvested dividends from paper Pair Trading.
    # Section 2.1 - Pairs Formation Page 11.
    # This means we do NOT include dividends as cumulative returns.
    ds_temp_a = df_temp_f['AdjClose_Normalized']
    
    # Name the Pandas Series
    ds_temp_a.name = ticker
    
    # Rename the column in the Pandas Series.
    ds_temp_a.columns = [ticker]
    
    # Append the Pandas Series to the list.
    list_of_series_for_software.append( ds_temp_a )

# Make Pandas DataFrame from Pandas Series.
df_ticker_closing_for_software = pd.concat(list_of_series_for_software, axis = 1)

#----------------------

# Going to concat multiple Panda series into a Panda DataFrame
list_of_series_for_semiconductors = []
for ticker in dp_tickers_for_semiconductors.items:
    
    df_temp_e = pd.DataFrame()
        
    #----------------------------------------
    
    # Getting adjusted close. It takes into account stock split and
    # dividend pay outs.
    # https://www.investopedia.com/terms/a/adjusted_closing_price.asp
    #
    # Where price includes reinvested dividends from paper Pair Trading.
    # Section 2.1 - Pairs Formation Page 11.
    # This means we do NOT include dividends as cumulative returns.
    df_temp_e['AdjClose'] = dp_tickers_for_semiconductors[ticker]['AdjClose']
    df_temp_e['ExDividend'] = dp_tickers_for_semiconductors[ticker]['ExDividend']
   
    # Forward fill then backward fill data.
    df_temp_e = df_temp_e.ffill().bfill()

    # Adjusted close normalized by dividing by the price of day 1 of the formation period.
    df_temp_e['AdjClose_Normalized']  = df_temp_e['AdjClose'] / df_temp_e.iloc[0]['AdjClose']
    
    # Get the daily returns
    # https://stackoverflow.com/questions/20000726/calculate-daily-returns-with-pandas-dataframe
    df_temp_e['daily_returns'] = df_temp_e['AdjClose'] - df_temp_e['AdjClose'].shift(1)
   
    # Fill all not a number (NaN) with 0.
    df_temp_e['daily_returns'] = df_temp_e['daily_returns'].fillna(0.0)

    # Get cumulative returns
    df_temp_e['cumulative_returns'] = df_temp_e['daily_returns'].cumsum()
    
    # Normalized cumulative returns
    df_temp_e['cumulative_returns_normalized'] = df_temp_e['cumulative_returns'] / df_temp_e.iloc[0]['AdjClose']
    
    # Add dividend pay out.
    df_temp_e['cumulative_returns_with_dividends'] = df_temp_e['cumulative_returns'] + df_temp_e['ExDividend']
    
    # Normalized cumulative returns by dividing by the price of day 1 of the formation period.
    df_temp_e['cumulative_returns_with_dividends_normalized'] = df_temp_e['cumulative_returns_with_dividends'] / df_temp_e.iloc[0]['AdjClose']
    
    # Forward fill then backward fill data.
    df_temp_e = df_temp_e.ffill().bfill()

    
    # Gather data for Debugging
    df_temp_f = df_temp_e.copy()
    dict_of_dataframes_modified_for_software[ticker] = df_temp_f
    
    # Copy only the Pandas Series
    #
    # Where price includes reinvested dividends from paper Pair Trading.
    # Section 2.1 - Pairs Formation Page 11.
    # This means we do NOT include dividends as cumulative returns.
    ds_temp_a = df_temp_f['AdjClose_Normalized']
    
    # Name the Pandas Series
    ds_temp_a.name = ticker
    
    # Rename the column in the Pandas Series.
    ds_temp_a.columns = [ticker]
        
    # Append the Pandas Series to the list.
    list_of_series_for_semiconductors.append( ds_temp_a )

# Make Pandas DataFrame from Pandas Series.
df_ticker_closing_for_semiconductors = pd.concat(list_of_series_for_semiconductors, axis = 1)

### Debug

In [290]:
df_ticker_closing_for_software

,ADBE,ADP,ADSK,AKAM,ANSS,ATVI,CA,CDNS,CRM,CTXS,...,NTAP,ORCL,PAYX,RHT,SNPS,SYMC,TSS,V,VRSN,WU
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-18,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2017-01-19,1.009192,1.000000,1.000000,0.991092,1.000000,0.994371,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2017-01-20,1.017649,1.002622,1.021130,0.993534,0.997968,0.997185,1.000614,0.998845,1.006087,1.003886,...,1.015071,1.016832,1.009360,1.003788,0.997702,1.003389,1.000937,1.001346,1.000498,0.992901
2017-01-23,1.020039,0.992522,1.019505,0.976580,1.000214,0.985926,0.996928,0.988448,1.007807,0.998057,...,1.009489,1.011987,1.003448,1.001894,0.998687,1.007530,0.994005,1.005139,0.997512,0.980596
2017-01-24,1.045317,1.002622,1.025256,0.973276,1.010589,1.000512,1.012289,1.010012,1.020378,1.018784,...,1.047167,1.022698,1.015928,1.020427,1.024626,1.029744,0.996441,1.018353,1.002115,0.966398
2017-01-25,1.050188,0.997378,1.033383,0.987500,1.015617,1.012282,0.961905,1.011167,1.038375,1.033574,...,1.059168,1.023973,1.011823,1.033956,1.029059,1.026355,0.968715,1.026551,1.011570,0.950308
2017-01-26,1.037595,0.990968,1.020755,0.973851,1.014547,1.009980,0.962826,0.999615,1.031494,0.966426,...,1.049400,1.023463,1.009031,1.017992,1.021179,1.024473,0.960097,1.018475,1.014556,0.927118
2017-01-27,1.047799,0.987763,1.022256,0.976580,1.004920,1.013562,0.956375,1.009626,1.034934,0.974414,...,1.053865,1.026014,1.004433,1.024080,1.026104,1.029744,0.960659,1.024960,1.010450,0.927118
2017-01-30,1.046236,0.990288,1.014754,0.974282,0.999893,1.013562,0.956375,1.005776,1.041551,0.983807,...,1.062517,1.026014,1.006076,1.021510,1.028074,1.026732,0.960472,1.024104,1.005723,0.928064


In [291]:
df_ticker_closing_for_semiconductors

,ADI,AMAT,AMD,AVGO,INTC,KLAC,LRCX,MCHP,MU,NVDA,QCOM,QRVO,SWKS,TXN,XLNX
Date,,,,,,,,,,,,,,,
2017-01-18,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2017-01-19,1.000000,1.000000,1.000000,1.000000,0.994831,1.000000,1.000000,1.000000,1.000000,1.000000,0.989406,1.000000,1.000000,1.000000,1.000000
2017-01-20,1.006074,1.002667,0.997953,1.029748,1.004897,1.005052,1.013623,1.018877,1.011515,0.989064,0.965454,1.046476,1.130130,1.011776,1.000862
2017-01-23,1.004694,1.000000,1.014330,1.029317,1.000272,1.011707,1.017412,1.020399,1.008291,0.999334,0.842622,1.060710,1.149248,1.025041,0.999655
2017-01-24,1.014771,1.008296,1.068577,1.065693,1.023395,1.020333,1.034284,1.033643,1.052510,1.020635,0.844465,1.071857,1.176013,1.043313,1.016126
2017-01-25,1.028990,1.019852,1.059365,1.091183,1.028292,1.043376,1.054854,1.047648,1.085214,1.025010,0.873637,1.098268,1.170915,1.063617,1.027251
2017-01-26,1.024296,1.006815,1.076766,1.100291,1.021763,1.025878,1.031487,1.029533,1.083372,1.042697,0.829879,1.086263,1.159954,1.072821,0.989479
2017-01-27,1.061982,1.038222,1.092119,1.108590,1.033188,1.061738,1.066763,1.046735,1.104099,1.062857,0.832796,1.091579,1.172699,1.062940,1.008796
2017-01-30,1.053009,1.017481,1.085977,1.095171,1.017954,1.062847,1.046554,1.051302,1.113772,1.046215,0.823123,1.102555,1.185190,1.061169,1.019317


### Minimized Sum of Squared Deviations within the Main Industry Only - Software

In [336]:
# https://stackoverflow.com/questions/39203662/euclidean-distance-matrix-using-pandas
#pd.DataFrame( , columns = df_ticker_closing_for_semiconductors.columns, index = df_ticker_closing_for_semiconductors.index)

# https://stackoverflow.com/questions/41337316/pandas-compare-all-dataframe-columns-with-eachother
df = df_ticker_closing_for_software.copy()

# This is a numpy array
ndarray_a = df.values
column_names = df.columns

# Multiplying every column by every other column
#list_for_matrix_a = []
dict_pairs = {}
for i in range(len(ndarray_a.T)):
    
    column_name_in_focus = column_names[i]
    column_values_in_focus = ndarray_a[:, i]
    
    # The other columns
    for j in range(len(ndarray_a.T)):
        
        # Skip comparison to itself 
        if j == i:
            continue
            
        column_name_other = column_names[j]
        column_values_other = ndarray_a[:, j]
        
#         value_a = column_values_in_focus.sum() - column_values_other.sum()
#         sum_squared_deviations = value_a**2
        
        # These are numpy ndarrays
        ndarray_temp_a = column_values_in_focus - column_values_other
        ndarray_temp_b = np.square(ndarray_temp_a)
        
        sum_squared_deviations = ndarray_temp_b.sum()

        # Put in a list so we can sort it.
        # That way, we will have unique keys in the dictionary.
        # ('ADI', 'XLNX') is the same as ('XLNX', 'ADI') in this case.
        # Yes, the value for the key will be overriden, but it would be the
        # same value.
        #
        # After convert to a tuple. No real reason I can think of except convention.
        list_pair_key = sorted([column_name_in_focus, column_name_other])
        tuple_pair_key = tuple(list_pair_key)
        
        dict_pairs[tuple_pair_key] = sum_squared_deviations

In [337]:
dict_pairs

{('ADBE', 'ADP'): 29.376129693592553,
 ('ADBE', 'ADSK'): 4.158609456332218,
 ('ADBE', 'AKAM'): 94.87267386981122,
 ('ADBE', 'ANSS'): 0.8696850922794794,
 ('ADBE', 'ATVI'): 6.517351640000458,
 ('ADBE', 'CA'): 36.511020170463276,
 ('ADBE', 'CDNS'): 1.2265447236409859,
 ('ADBE', 'CRM'): 7.085918839247967,
 ('ADBE', 'CTXS'): 23.967198766619457,
 ('ADBE', 'EA'): 6.204815344227672,
 ('ADBE', 'EBAY'): 16.506262595260367,
 ('ADBE', 'FB'): 4.966219754693422,
 ('ADBE', 'FIS'): 18.847753171034842,
 ('ADBE', 'FISV'): 19.019434180821282,
 ('ADBE', 'GOOG'): 12.213583359818005,
 ('ADBE', 'GOOGL'): 13.570674681293951,
 ('ADBE', 'INTU'): 9.254211425211263,
 ('ADBE', 'MA'): 7.418980673896577,
 ('ADBE', 'MSFT'): 8.58283144569894,
 ('ADBE', 'NFLX'): 6.259575001444054,
 ('ADBE', 'NTAP'): 7.682165352235385,
 ('ADBE', 'ORCL'): 10.948462323843168,
 ('ADBE', 'PAYX'): 35.64310708952115,
 ('ADBE', 'RHT'): 1.153945770702228,
 ('ADBE', 'SNPS'): 3.811917449321886,
 ('ADBE', 'SYMC'): 23.828023833314973,
 ('ADBE', 'T

In [338]:
# Sort dictionary by value
d = dict_pairs

# Note: Dictionaries prior to Python 3.6 (I believe,) are not ordered.
#       So returning in 
list_minimized_sum_of_squared_deviations = [(k, d[k]) for k in sorted(d, key=d.get, reverse=False)]
list_minimized_sum_of_squared_deviations

[(('GOOG', 'GOOGL'), 0.04045301814067839),
 (('CRM', 'V'), 0.2072181498473113),
 (('V', 'VRSN'), 0.27237885594731637),
 (('FIS', 'FISV'), 0.2857409476522602),
 (('INTU', 'MSFT'), 0.2961582963157633),
 (('CRM', 'MSFT'), 0.36001117965462753),
 (('MA', 'VRSN'), 0.36546272129588436),
 (('MA', 'V'), 0.39234837790166055),
 (('MA', 'MSFT'), 0.39764108964701733),
 (('MA', 'TSS'), 0.41429499358425526),
 (('MSFT', 'V'), 0.4652088816480763),
 (('CRM', 'VRSN'), 0.4678096232573099),
 (('FB', 'V'), 0.4706843344746586),
 (('CRM', 'INTU'), 0.5105766639488711),
 (('FB', 'VRSN'), 0.5305884543622407),
 (('GOOG', 'INTU'), 0.5793654361769872),
 (('EBAY', 'FISV'), 0.5815084586772623),
 (('MSFT', 'TSS'), 0.5873446524515048),
 (('INTU', 'MA'), 0.5980149073455097),
 (('INTU', 'V'), 0.602668214336919),
 (('EBAY', 'FIS'), 0.6089873412974444),
 (('MSFT', 'VRSN'), 0.612903633991063),
 (('MA', 'NFLX'), 0.6369017056832544),
 (('CRM', 'MA'), 0.6511549699849806),
 (('FB', 'SNPS'), 0.6825352026638856),
 (('GOOG', 'MSFT

### Minimized Sum of Squared Deviations within the Related Industry Only - Semiconductors

In [339]:
# https://stackoverflow.com/questions/39203662/euclidean-distance-matrix-using-pandas
#pd.DataFrame( , columns = df_ticker_closing_for_semiconductors.columns, index = df_ticker_closing_for_semiconductors.index)

# https://stackoverflow.com/questions/41337316/pandas-compare-all-dataframe-columns-with-eachother
df = df_ticker_closing_for_semiconductors.copy()

# This is a numpy array
ndarray_a = df.values
column_names = df.columns

# Multiplying every column by every other column
#list_for_matrix_a = []
dict_pairs = {}
for i in range(len(ndarray_a.T)):
    
    column_name_in_focus = column_names[i]
    column_values_in_focus = ndarray_a[:, i]
    
    # The other columns
    for j in range(len(ndarray_a.T)):
        
        # Skip comparison to itself 
        if j == i:
            continue
            
        column_name_other = column_names[j]
        column_values_other = ndarray_a[:, j]
        
        # These are numpy ndarrays
        ndarray_temp_a = column_values_in_focus - column_values_other
        ndarray_temp_b = np.square(ndarray_temp_a)
        
        sum_squared_deviations = ndarray_temp_b.sum()

        # Put in a list so we can sort it.
        # That way, we will have unique keys in the dictionary.
        # ('ADI', 'XLNX') is the same as ('XLNX', 'ADI') in this case.
        # Yes, the value for the key will be overriden, but it would be the
        # same value.
        #
        # After convert to a tuple. No real reason I can think of except convention.
        list_pair_key = sorted([column_name_in_focus, column_name_other])
        tuple_pair_key = tuple(list_pair_key)
        
        dict_pairs[tuple_pair_key] = sum_squared_deviations


In [340]:
dict_pairs

{('ADI', 'AMAT'): 15.409621894273716,
 ('ADI', 'AMD'): 10.270124812815467,
 ('ADI', 'AVGO'): 7.410060308379357,
 ('ADI', 'INTC'): 3.728934838622128,
 ('ADI', 'KLAC'): 2.2691757219751896,
 ('ADI', 'LRCX'): 33.62975145123157,
 ('ADI', 'MCHP'): 4.393596211924288,
 ('ADI', 'MU'): 53.50393464508413,
 ('ADI', 'NVDA'): 52.403051521260295,
 ('ADI', 'QCOM'): 18.563161902462408,
 ('ADI', 'QRVO'): 2.0794069574152445,
 ('ADI', 'SWKS'): 6.976968857740386,
 ('ADI', 'TXN'): 1.9859323681800614,
 ('ADI', 'XLNX'): 0.8472312767165687,
 ('AMAT', 'AMD'): 20.67896462610036,
 ('AMAT', 'AVGO'): 3.7710241391412676,
 ('AMAT', 'INTC'): 26.80535021389236,
 ('AMAT', 'KLAC'): 8.348660336741,
 ('AMAT', 'LRCX'): 3.7335933812111457,
 ('AMAT', 'MCHP'): 4.27285769584317,
 ('AMAT', 'MU'): 13.121547700573586,
 ('AMAT', 'NVDA'): 13.855923845250377,
 ('AMAT', 'QCOM'): 61.494638709091895,
 ('AMAT', 'QRVO'): 13.435834239448647,
 ('AMAT', 'SWKS'): 7.672281410194768,
 ('AMAT', 'TXN'): 10.236713850295295,
 ('AMAT', 'XLNX'): 15.2

In [341]:
# Sort dictionary by value
d = dict_pairs

# Note: Dictionaries prior to Python 3.6 (I believe,) are not ordered.
#       So returning in 
list_minimized_sum_of_squared_deviations = [(k, d[k]) for k in sorted(d, key=d.get, reverse=False)]
list_minimized_sum_of_squared_deviations

[(('ADI', 'XLNX'), 0.8472312767165687),
 (('AVGO', 'MCHP'), 1.234849025424745),
 (('KLAC', 'MCHP'), 1.59847575685719),
 (('AVGO', 'SWKS'), 1.822496536249247),
 (('KLAC', 'QRVO'), 1.8528020213566363),
 (('ADI', 'TXN'), 1.9859323681800614),
 (('ADI', 'QRVO'), 2.0794069574152445),
 (('ADI', 'KLAC'), 2.2691757219751896),
 (('TXN', 'XLNX'), 2.4415248210604705),
 (('MCHP', 'SWKS'), 2.457283560344462),
 (('KLAC', 'TXN'), 2.5557861098371353),
 (('KLAC', 'SWKS'), 2.8486200736535103),
 (('QRVO', 'XLNX'), 2.881268035181991),
 (('KLAC', 'XLNX'), 2.9232627425123208),
 (('QRVO', 'SWKS'), 2.9291445432155454),
 (('AVGO', 'KLAC'), 3.004284756473002),
 (('MCHP', 'TXN'), 3.3422108791541403),
 (('MCHP', 'QRVO'), 3.5434010166077172),
 (('INTC', 'XLNX'), 3.595061680411969),
 (('ADI', 'INTC'), 3.728934838622128),
 (('AMAT', 'LRCX'), 3.7335933812111457),
 (('AMAT', 'AVGO'), 3.7710241391412676),
 (('AMAT', 'MCHP'), 4.27285769584317),
 (('ADI', 'MCHP'), 4.393596211924288),
 (('MCHP', 'XLNX'), 4.531357897873351)

### Minimized Sum of Squared Deviations between Main industry and Related industry - Software and Semiconductors

In [342]:
# https://stackoverflow.com/questions/39203662/euclidean-distance-matrix-using-pandas
#pd.DataFrame( , columns = df_ticker_closing_for_semiconductors.columns, index = df_ticker_closing_for_semiconductors.index)

# https://stackoverflow.com/questions/41337316/pandas-compare-all-dataframe-columns-with-eachother
df_a = df_ticker_closing_for_software.copy()
df_b = df_ticker_closing_for_semiconductors.copy()

# This is a numpy array
ndarray_a = df_a.values
column_names_a = df_a.columns
#
ndarray_b = df_b.values
column_names_b = df_b.columns

# Multiplying every column by every other column
#list_for_matrix_a = []
dict_pairs = {}
for i in range(len(ndarray_a.T)):
    
    column_name_in_focus = column_names_a[i]
    column_values_in_focus = ndarray_a[:, i]
    
    # The other columns
    for j in range(len(ndarray_b.T)):
        
#         # Skip comparison to itself 
#         if j == i:
#             continue
            
        column_name_other = column_names_b[j]
        column_values_other = ndarray_b[:, j]
        
        # These are numpy ndarrays
        ndarray_temp_a = column_values_in_focus - column_values_other
        ndarray_temp_b = np.square(ndarray_temp_a)
        
        sum_squared_deviations = ndarray_temp_b.sum()

        # Put in a list so we can sort it.
        # That way, we will have unique keys in the dictionary.
        # ('ADI', 'XLNX') is the same as ('XLNX', 'ADI') in this case.
        # Yes, the value for the key will be overriden, but it would be the
        # same value.
        #
        # After convert to a tuple. No real reason I can think of except convention.
        #list_pair_key = sorted([column_name_in_focus, column_name_other])
        tuple_pair_key = (column_name_in_focus, column_name_other)
        
        dict_pairs[tuple_pair_key] = sum_squared_deviations


In [343]:
dict_pairs

{('ADBE', 'ADI'): 15.641497452290585,
 ('ADBE', 'AMAT'): 1.2800980451057407,
 ('ADBE', 'AMD'): 21.141148492516677,
 ('ADBE', 'AVGO'): 3.229031816092198,
 ('ADBE', 'INTC'): 26.603769805898253,
 ('ADBE', 'KLAC'): 8.983868763624907,
 ('ADBE', 'LRCX'): 4.947437744266467,
 ('ADBE', 'MCHP'): 4.7534664617861475,
 ('ADBE', 'MU'): 14.015114483544183,
 ('ADBE', 'NVDA'): 14.090597686215157,
 ('ADBE', 'QCOM'): 60.97321968260145,
 ('ADBE', 'QRVO'): 13.91965734214309,
 ('ADBE', 'SWKS'): 8.013527852182303,
 ('ADBE', 'TXN'): 9.743496085765168,
 ('ADBE', 'XLNX'): 15.995232558511132,
 ('ADP', 'ADI'): 3.3399481439441305,
 ('ADP', 'AMAT'): 29.55695293245271,
 ('ADP', 'AMD'): 19.534769843996934,
 ('ADP', 'AVGO'): 18.21057535276634,
 ('ADP', 'INTC'): 1.513227017157409,
 ('ADP', 'KLAC'): 9.546610453832688,
 ('ADP', 'LRCX'): 52.833379866900344,
 ('ADP', 'MCHP'): 13.297060351310222,
 ('ADP', 'MU'): 77.29454456617162,
 ('ADP', 'NVDA'): 72.71541042059124,
 ('ADP', 'QCOM'): 7.667910811503187,
 ('ADP', 'QRVO'): 7.

In [344]:
# Sort dictionary by value
d = dict_pairs

# Note: Dictionaries prior to Python 3.6 (I believe,) are not ordered.
#       So returning in 
list_minimized_sum_of_squared_deviations = [(k, d[k]) for k in sorted(d, key=d.get, reverse=False)]
list_minimized_sum_of_squared_deviations

[(('GOOGL', 'XLNX'), 0.5245130065173396),
 (('MSFT', 'TXN'), 0.5759653537096523),
 (('SNPS', 'MCHP'), 0.5986578446501867),
 (('FB', 'MCHP'), 0.6189008071597084),
 (('FIS', 'XLNX'), 0.6465886000734158),
 (('EBAY', 'ADI'), 0.6669700061807349),
 (('GOOG', 'XLNX'), 0.6938522819778221),
 (('FISV', 'XLNX'), 0.7275039920196817),
 (('FISV', 'ADI'), 0.8126773010290321),
 (('V', 'MCHP'), 0.9116461786887379),
 (('EBAY', 'XLNX'), 0.9172979723184711),
 (('SNPS', 'AVGO'), 0.9195603825634844),
 (('FIS', 'ADI'), 0.9721460842565901),
 (('GOOGL', 'ADI'), 0.9991659273680565),
 (('MA', 'TXN'), 1.0191323715355078),
 (('INTU', 'TXN'), 1.0660962802519576),
 (('CRM', 'TXN'), 1.0856977608231528),
 (('GOOG', 'ADI'), 1.1283331094134528),
 (('TSS', 'TXN'), 1.1467625205748755),
 (('CRM', 'MCHP'), 1.20869169961224),
 (('VRSN', 'MCHP'), 1.2180349493319742),
 (('V', 'TXN'), 1.2447344545367391),
 (('ADBE', 'AMAT'), 1.2800980451057407),
 (('CRM', 'KLAC'), 1.3086781125145386),
 (('FB', 'AVGO'), 1.3819220813233803),
 (('

In [347]:
len(list_minimized_sum_of_squared_deviations)

465

### Excess Return Computation
Because pairs may open and close at various points during the six-month trading period,
the calculation of the excess return on a portfolio of pairs is a non-trivial issue. Pairs that open
and converge during the trading interval will have positive cash flows. Because pairs can reopen
after initial convergence, they can have multiple positive cash flows during the trading
interval. Pairs that open but do not converge will only have cash flows on the last day of the
trading interval when all positions are closed out. Therefore, the payoffs to pairs trading
strategies are a set of positive cash flows that are randomly distributed throughout the trading
period, and a set of cash flows at the end of the trading interval which can either be positive or
negative.

#### Main Industry (Software)
(('GOOG', 'GOOGL'), 0.04045301814067839)
#### Main Industry and Related Industry (Software and Semiconductors)
(('GOOGL', 'XLNX'), 0.5245130065173396)


### Get the daily returns for these industries (GOOG, GOOGL, XLNX)

In [116]:
# Get just the column for the normalized price (which is of type Pandas Series)
# and convert to Pandas DataFrame.
df_goog = df_ticker_closing_for_software['GOOG'].to_frame()
df_googl = df_ticker_closing_for_software['GOOGL'].to_frame()
df_xlnx = df_ticker_closing_for_semiconductors['XLNX'].to_frame()

# Change the names of the columns.
df_goog.columns = ['normalized_price']
df_googl.columns = ['normalized_price']
df_xlnx.columns = ['normalized_price']

# Calculate the mean average
df_goog_mean_average = df_goog['normalized_price'].mean()
df_googl_mean_average = df_goog['normalized_price'].mean()
df_xlnx_mean_average = df_goog['normalized_price'].mean()

# daily returns
# https://www.fool.com/knowledge-center/how-to-convert-daily-returns-to-annual-returns.aspx
# amount
df_goog['daily_return'] = (df_goog['normalized_price'] / df_goog['normalized_price'][0]) - 1.0
df_googl['daily_return'] = (df_googl['normalized_price'] / df_googl['normalized_price'][0]) - 1.0
df_xlnx['daily_return'] = (df_xlnx['normalized_price'] / df_xlnx['normalized_price'][0]) - 1.0

# Average return
df_goog_average_return = df_goog['daily_return'].mean()
df_googl_average_return = df_googl['daily_return'].mean()
df_xlnx_average_return = df_xlnx['daily_return'].mean()

# Standard Deviation of return
goog_std_return= df_goog['daily_return'].std()
googl_std_return = df_googl['daily_return'].std()
xlnx_std_return = df_xlnx['daily_return'].std()

# Get the mean of pairs
#
# Main Industry pairs
mean_pair_goog_googl = (df_goog['normalized_price'] + df_googl['normalized_price']).mean()
mean_pair_googl_googl = (df_googl['normalized_price'] + df_goog['normalized_price']).mean()
#
# Related Industry pairs
mean_pair_googl_xlnx = (df_googl['normalized_price'] + df_xlnx['normalized_price']).mean()
mean_pair_xlnx_googl = (df_xlnx['normalized_price'] + df_googl['normalized_price']).mean()

# Get the standard deviation between pairs
#
# Main Industry pairs
std_pair_goog_googl = (df_goog['normalized_price'] + df_googl['normalized_price']).std()
std_pair_googl_googl = (df_googl['normalized_price'] + df_goog['normalized_price']).std()
#
# Related Industry pairs
std_pair_googl_xlnx = (df_googl['normalized_price'] + df_xlnx['normalized_price']).std()
std_pair_xlnx_googl = (df_xlnx['normalized_price'] + df_googl['normalized_price']).std()

# Find whenever the normalized price was 2 times over the standard deviation.
#df_temp_g = df_xlnx[np.abs(df_xlnx['normalized_price'] - mean_pair_googl_xlnx) > 2.0*std_pair_googl_xlnx]

# Holding period return
df_goog_holding_period_return =  (df_goog.iloc[-1] / df_goog.iloc[0]) - 1
df_googl_holding_period_return =  (df_googl.iloc[-1] / df_googl.iloc[0]) - 1
df_xlnx_holding_period_return =  (df_xlnx.iloc[-1] / df_xlnx.iloc[0]) - 1

# Sharpe measure
#
# Sharpe measure is on total risk.
# Sharpe Raito = (Rp - Rf)/std
# Rp => Average return
# Rf => Risk free rate
# std => standard deviation
#
# https://www.investopedia.com/terms/r/risk-freerate.asp#ixzz55B5Vc4ly
# the interest rate on a three-month U.S. Treasury bill is often used as the risk-free rate for U.S.-based investors.
#
# Risk free rate: https://ycharts.com/indicators/3_month_t_bill
Rp = df_goog_average_return
Rf = 0.0133
std = goog_std_return
sharpe_goog = (Rp - Rf) / std
#
Rp = df_googl_average_return
Rf = 0.0133
std = googl_std_return
sharpe_googl = (Rp - Rf) / std
#
Rp = df_xlnx_average_return
Rf = 0.0133
std = xlnx_std_return
sharpe_xlnx = (Rp - Rf) / std

# Treynor measure
#
# Similar to Sharpe except it only considers systematic risks
# not total risk.
#
# Treynor Raito = (Rp - Rf)/Beta
# Rp => Average return
# Rf => Risk free rate
# Beta => Beta
#
# https://www.investopedia.com/terms/r/risk-freerate.asp#ixzz55B5Vc4ly
# the interest rate on a three-month U.S. Treasury bill is often used as the risk-free rate for U.S.-based investors.
#
# Risk free rate: https://ycharts.com/indicators/3_month_t_bill
#
# Beta - https://finance.yahoo.com/quote/GOOG?p=GOOG
#        https://finance.yahoo.com/quote/GOOGL?p=GOOGL  
Rp = df_goog_average_return
Rf = 0.0133
Beta = 1.04
treynor_goog = (Rp - Rf) / Beta 
#
Rp = df_googl_average_return
Rf = 0.0133
Beta = 1.01
treynor_googl = (Rp - Rf) / Beta 
#
Rp = df_goog_average_return
Rf = 0.0133
Beta = 0.88
treynor_xlnx = (Rp - Rf) / Beta 


# Jensen's Measure
#
# Excess returns
# alpha = Rp - [Rf - Beta(Rm - Rf)]
# Rm is Market return. The market is the S&P 500 so using ticker symbol 
# SPX which follows the S&P 500.
df_spx['daily_return'] = (df_spx['Adj Close'] / (df_spx['Adj Close'][1]))
df_spx_average_return = df_spx['daily_return'].mean()
#
Rp = df_goog_average_return
Rf = 0.0133
Beta = 0.88
Rm = df_spx_average_return
goog_jensen_alpha = Rp - (Rf - Beta*(Rm - Rf))
#
Rp = df_googl_average_return
Rf = 0.0133
Beta = 0.88
Rm = df_spx_average_return
googl_jensen_alpha = Rp - (Rf - Beta*(Rm - Rf))
#
Rp = df_xlnx_average_return
Rf = 0.0133
Beta = 0.88
Rm = df_spx_average_return
xlnx_jensen_alpha = Rp - (Rf - Beta*(Rm - Rf))

In [117]:
goog_jensen_alpha

0.882301658875199

In [118]:
googl_jensen_alpha

0.8719089729169318

In [119]:
xlnx_jensen_alpha

0.8544309763096805

### DEBUG

In [59]:
df_goog_holding_period_return

normalized_price    0.418032
daily_return             inf
dtype: float64

In [53]:
df_googl_holding_period_return

normalized_price    0.38712
dtype: float64

In [54]:
df_xlnx_holding_period_return

normalized_price    0.32766
dtype: float64

In [66]:
sharpe_goog

1.4782398792731872

In [68]:
sharpe_googl

1.4620040368557

In [69]:
sharpe_xlnx

1.2894166002250353

In [71]:
treynor_goog

0.14717192208828575

In [72]:
treynor_googl

0.14125357724113863

In [73]:
treynor_xlnx

0.17393045337706498